# 파인튜닝된 모델 성능 확인
- LoRA 파인튜닝이 끝난 모델의 성능을 확인해 보자.

## 원본 모델 로드
- 파인튜닝 되기 전 원본 모델 부분을 불러온다.
- LoRA의 weight는 어떻게 가져와야 할까? 모르겠다. 알아보자.
- beomi 선생님 고마워요! 추가되었군.

In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from peft import PeftModel, PeftConfig

# model_id = "EleutherAI/polyglot-ko-12.8b"
model_id = "beomi/KoAlpaca-Polyglot-12.8B"
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype=torch.bfloat16
)

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config=bnb_config, device_map={"":0})


In [4]:
model = PeftModel.from_pretrained(model, '/home/nas4/user/kjh4/SogangAlpaca/outputs/checkpoint-45000')

In [5]:
model.eval()
model.config.use_cache = True  # silence the warnings. Please re-enable for inference!

In [6]:
input_tokens = tokenizer("### 질문: 등록금 납부하려면 어디로 가야 해?", return_tensors='pt')
input_ids = input_tokens.input_ids.to(0)
input_ids

tensor([[    6,     6,     6,  2438,    29,  7586,  6472,   284,  2604,  2116,
           286, 15983,   547,    34]], device='cuda:0')

In [9]:
def gen(x):
    gened = model.generate(
        **tokenizer(
            f"### 질문: {x}\n\n### 답변:", 
            return_tensors='pt', 
            return_token_type_ids=False
        ).to(0), 
        max_new_tokens=256,
        early_stopping=True,
        do_sample=True,
        eos_token_id=2,
    )
    print(tokenizer.decode(gened[0]))

In [10]:
gen('건강하게 살기 위한 세 가지 방법은?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 건강하게 살기 위한 세 가지 방법은?

### 답변: 건강하게 살기 위해서는 올바른 식습관, 규칙적인 운동, 정기적인 건강 검진이 필요합니다. 이는 건강한 삶을 유지하기 위해 기본적인 요소이며, 각각 중요성과 대체할 수 없는 개성이 있습니다. 

- 올바른 식습관: 균형 잡힌 식습관은 건강한 삶을 위해 가장 기초적인 것입니다. 탄수화물, 단백질, 지방, 비타민, 무기질 등 영양소를 충분히 섭취해야 하며, 간식 대신 식사를 규칙적으로 하는 것이 좋습니다. 

- 규칙적인 운동: 운동은 스트레스를 감소시켜 건강한 삶을 유지하는 데 중요한 역할을 합니다. 걷기, 조깅, 자전거 타기 등 유산소 운동을 추천합니다. 

- 정기적인 건강 검진: 건강에 이상이 생기기 전에 미리 검진을 받아서 몸 상태를 체크하고, 필요한 경우 의학적 조치를 취하는 것이 중요합니다. 

물론, 건강한 삶을 사는 세 가지 방법은 서로 큰 연관성이 있습니다. 균형 잡힌 식습관은 규칙적인 운동을 추천하며, 규칙적인 운동은 정기적인 건강 검진이 필요로 합니다. 따라서 건강한 삶을 위해서는 세


In [11]:
gen('서강대학교 등록금 납부하려면 어디로 가야 해?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 서강대학교 등록금 납부하려면 어디로 가야 해?

### 답변: 서강대의 등록금 고지서 출력이나 등록금 납부는 아래의 경로를 통해 진행할 수 있습니다. 

1. 고지서 출력 - 온라인으로 고지서를 출력하려면, 고지서에 기재된 전용 계좌로 등록금을 이체해야 합니다. 현금 영수증이 필요하다면 현금 영수증 등록을 따로 해야 합니다. 

2. 등록금 납부 - 은행에서 고지서에 쓰인 계좌번호로 등록금을 이체하면 됩니다. 또한, 등록금 수납내역 조회 및 등록금액 결제 기능을 이용해 등록금을 결제할 수도 있습니다. 

위의 방법을 통해 서강대학교 등록금을 납부하실 수 있습니다. <|endoftext|>


In [12]:
gen('익스트림 수요미식회 빌런 찾습니다')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 익스트림 수요미식회 빌런 찾습니다

### 답변:해당 질문을 찾지 못했습니다. 익스트림 수요미식회에 등장하는 인물들은 모두 미식가이지만, 빌런이 등장하는 장면은 나오지 않습니다. 혹시 해당 내용을 어디에서 다루었는지 알려주시면 추가로 찾아보겠습니다. <|endoftext|>


In [13]:
gen('주변에 7학기하고 휴학하시는 분들 많나요?')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 주변에 7학기하고 휴학하시는 분들 많나요?

### 답변:네, 생각보다 많이 있습니다. 학교마다 상이할 수 있지만 보통 매 학기마다 15-20명 정도의 휴학생이 발생하고 있습니다. 7학기까지 수료한 후 휴학을 하는 분들도 있지만, 7학기 수료 전에 휴학하는 분들도 있습니다. 그 이유는각 개인마다 다르지만, 보통은 학업과 일상 생활의 균형을 맞추기 위해서 또는 더 많은 경험을 쌓기 위해 휴학을 하는 경우가 많습니다. <|endoftext|>


In [14]:
gen('바닥부터 영상 배워보고 싶은데 현실적인 조언 부탁드립니다.,')

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


### 질문: 바닥부터 영상 배워보고 싶은데 현실적인 조언 부탁드립니다.,

### 답변:영상 제작 분야에서 일을 하고 싶다면, 관련된 기술과 지식을 배우고 경험을 쌓아야 합니다. 이를 위해 다음과 같은 방법을 추천합니다.
1. 대학에서 전문적인 공부를 해보세요. 관련된 전공이 있는 영상디자인, 영상학, 촬영 등의 학과를 알아보시고, 신문방송학과나 예술학과 같은 관련된 학과에서도 많은 공부를 할 수 있습니다.
2. 영상 업체에서 일을 해보세요. 작은 보조 업무부터 시작해서 정규직으로 일할 수 있는 기회가 있습니다.
3. 영상 제작 교육을 받아보세요. 관련된 전문 교육기관에서 공부하고 경험을 쌓으실 수 있습니다.
4. 열정적인 전문가를 만나보세요. 열정적으로 영상 제작을 하시는 전문가들과 함께 일하며 배울 수 있습니다.
현실적인 조언을 드리면, 관련된 교육과 경험을 최대한 빠르게 해보시는 것이 좋습니다. 졸업 후 바로 취업해서 일하실 수 있는 상황이 된다면 그렇게 하는 것이 좋겠지만, 보통은 그렇지 않은 경우가 많을 것입니다. 그러므로 빠르게 관련 경험과 공부를 해보시
